In [15]:
import pandas as pd
import numpy as np

In [45]:
import pandas as pd
import numpy as np

# Load the data and replace '.' with NaN
SOFR_index_df = pd.read_csv('SOFR.csv')
SOFR_index_df['SOFR'] = SOFR_index_df['SOFR'].replace('.', np.nan)

# Convert SOFR column to float
SOFR_index_df['SOFR'] = SOFR_index_df['SOFR'].astype(float)

# Calculate the 'multiple' column
SOFR_index_df['multiple'] = (1 + SOFR_index_df['SOFR'] / 100) ** (1/360)

# Convert the 'DATE' column to datetime
SOFR_index_df['DATE'] = pd.to_datetime(SOFR_index_df['DATE'])

# Generate a full date range
full_date_range = pd.date_range(start=SOFR_index_df['DATE'].min(), end=SOFR_index_df['DATE'].max())

# Reindex the DataFrame to include all dates
SOFR_index_df = SOFR_index_df.set_index('DATE').reindex(full_date_range).reset_index()
SOFR_index_df.columns = ['DATE', 'SOFR', 'multiple']

# Create the 'business_day' column: 0 if SOFR is NaN, 1 if SOFR is not NaN
SOFR_index_df['business_day'] = np.where(SOFR_index_df['SOFR'].isnull(), 0, 1)

# Replace all NaN values in 'SOFR' with the last available business rate
SOFR_index_df['SOFR'].ffill(inplace=True)
SOFR_index_df['multiple'].ffill(inplace=True)

# Initialize the index_approx column with NaN
SOFR_index_df['index_approx'] = np.nan

# Set the initial value for index_approx on 2019-08-29
initial_date = pd.to_datetime('2019-08-29')
initial_value = 1.0316827

# Set the initial value for the first row matching the initial_date
SOFR_index_df.loc[SOFR_index_df['DATE'] == initial_date, 'index_approx'] = initial_value

# Calculate the subsequent values of index_approx
for i in range(1, len(SOFR_index_df)):
    if pd.notnull(SOFR_index_df.loc[i, 'multiple']):
        SOFR_index_df.loc[i, 'index_approx'] = SOFR_index_df.loc[i-1, 'index_approx'] * SOFR_index_df.loc[i, 'multiple']

SOFR_index_df['calendar_days_applicable'] = 0  # Default value

# Loop through and set calendar_days_applicable based on business_day
for i in range(len(SOFR_index_df) - 1):
    if SOFR_index_df.loc[i, 'business_day'] == 1:
        count = 1
        for j in range(i + 1, len(SOFR_index_df)):
            if SOFR_index_df.loc[j, 'business_day'] == 0:
                count += 1
            else:
                break
        SOFR_index_df.loc[i, 'calendar_days_applicable'] = count

# # Calculate the subsequent values of index_approx
# for i in range(1, len(SOFR_index_df)):
#     if pd.notnull(SOFR_index_df.loc[i, 'multiple_2']):
#         SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i-1, 'index_approx_2'] * SOFR_index_df.loc[i, 'multiple_2']

SOFR_index_df['multiple_2'] = 1 + (SOFR_index_df['SOFR'] / 100) * (SOFR_index_df['calendar_days_applicable'] / 360)


# Initialize the index_approx column with NaN
SOFR_index_df['index_approx_2'] = np.nan

# Set the initial value for index_approx on 2019-08-29
initial_date = pd.to_datetime('2019-08-29')
initial_value = 1.0316827

# Set the initial value for the first row matching the initial_date
SOFR_index_df.loc[SOFR_index_df['DATE'] == initial_date, 'index_approx_2'] = initial_value

for i in range(1, len(SOFR_index_df)):
    if SOFR_index_df.loc[i, 'business_day'] == 1:  # Only update on business days
        # Ensure the previous value exists before multiplication
        if pd.notnull(SOFR_index_df.loc[i-1, 'index_approx_2']):
            SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i-1, 'index_approx_2'] * SOFR_index_df.loc[i, 'multiple_2']
    else:
        SOFR_index_df.loc[i, 'index_approx_2'] = SOFR_index_df.loc[i - 1, 'index_approx_2']


# Display the updated DataFrame
SOFR_index_df.head(20)
SOFR_index_df.to_csv('SOFR_index.csv', index=False)
